In [3]:
#|default_exp build_lib

In [11]:
#|export
from fastcore.all import *

import pprint


# from json import loads
from jsonref import loads
import yaml
from collections import namedtuple

# Internal - OpenAPI Parser

This library leverages the [OpenAPI Specification](https://github.com/OAI/OpenAPI-Specification) to create a python client for the GitHub API.  The OpenAPI specification contains metadata on all of the endpoints and how to access them properly.  Using this metadata, we can construct a python client dynamically that updates automatically along with the OpenAPI Spec. 

In [12]:
#|export
GH_OPENAPI_URL = 'https://raw.githubusercontent.com/unitycatalog/unitycatalog/main/api/all.yaml'
_DOC_URL = 'https://docs.github.com/'

In [13]:
#|hide
if 0:
    s = urlread(GH_OPENAPI_URL)
    js = loads(s)['paths']
    sj = {o['operationId']:o for p in js.values() for o in p.values()}

    j = js['/repos/{owner}/{repo}/issues/{issue_number}/labels']['post']
    n = nested_idx(j, *'requestBody content application/json schema'.split())

In [14]:
#|export
_lu_type = dict(zip(
    'NA string object array boolean number integer'.split(),
    map(PrettyString,'object str dict list bool int int'.split())
))

def _detls(k,v):
    res = [_lu_type[v.get('type', 'NA')]]
    try: res.append(v['default'])
    except KeyError: pass
    return [k]+res

def _find_data(d):
    if 'properties' in d: return d['properties']
    if 'oneOf' in d:
        for o in d['oneOf']:
            if 'properties' in o: return o['properties']
    return {}

In [15]:
#|export
def build_funcs(nm='ghapi/metadata.py', url=GH_OPENAPI_URL, docurl=_DOC_URL):
    "Build module metadata.py from an Open API spec and optionally filter by a path `pre`"
    def _get_detls(o):
        data = nested_idx(o, *'requestBody content application/json schema'.split()) or {}
        data = _find_data(data)
        url = o['externalDocs']['url'][len(docurl):]
        params = o.get('parameters',None)
        qparams = [p['name'] for p in params if p['in']=='query'] if params else []
        d = [_detls(*o) for o in data.items()]
        preview = nested_idx(o, 'x-github','previews',0,'name') or ''
        return (o['operationId'], o['summary'], url, qparams, d, preview)

    print(url)
    js = yaml.safe_load(urlread(url))
    _funcs = [(path, verb)
              # + _get_detls(detls)
              for path,verbs in js['paths'].items() for verb,detls in verbs.items()
            #   if 'externalDocs' in detls
              ]
    Path(nm).write_text("funcs = " + pprint.pformat(_funcs, width=360))

In [16]:
#|hide
build_funcs(nm="unitycatalog_api/metadata.py")

https://raw.githubusercontent.com/unitycatalog/unitycatalog/main/api/all.yaml


In [17]:
url = "https://raw.githubusercontent.com/unitycatalog/unitycatalog/main/api/all.yaml"
js = yaml.safe_load(urlread(url))


In [21]:
_funcs = [(path, verb)
        #   + _get_detls(detls)
              for path,verbs in js['paths'].items() for verb,detls in verbs.items()
            #   if 'externalDocs' in detls
            ]

In [22]:
_funcs

[('/catalogs', 'post'),
 ('/catalogs', 'get'),
 ('/catalogs/{name}', 'parameters'),
 ('/catalogs/{name}', 'get'),
 ('/catalogs/{name}', 'patch'),
 ('/catalogs/{name}', 'delete'),
 ('/schemas', 'post'),
 ('/schemas', 'get'),
 ('/schemas/{full_name}', 'parameters'),
 ('/schemas/{full_name}', 'get'),
 ('/schemas/{full_name}', 'patch'),
 ('/schemas/{full_name}', 'delete'),
 ('/tables', 'post'),
 ('/tables', 'get'),
 ('/tables/{full_name}', 'parameters'),
 ('/tables/{full_name}', 'get'),
 ('/tables/{full_name}', 'delete'),
 ('/volumes', 'post'),
 ('/volumes', 'get'),
 ('/volumes/{name}', 'parameters'),
 ('/volumes/{name}', 'get'),
 ('/volumes/{name}', 'patch'),
 ('/volumes/{name}', 'delete'),
 ('/temporary-table-credentials', 'post'),
 ('/temporary-volume-credentials', 'post'),
 ('/functions', 'post'),
 ('/functions', 'get'),
 ('/functions/{name}', 'parameters'),
 ('/functions/{name}', 'get'),
 ('/functions/{name}', 'delete')]

This module created by `build_funcs` contains a list of metadata for each endpoint, containing the path, verb, operation id, summary, documentation relative URL, and list of parameters (if any), e.g:

In [24]:
#|export
UCMeta = namedtuple('UCMeta', 'path verb'.split())

In [25]:
from unitycatalog_api.metadata import funcs

In [26]:
UCMeta(*funcs[3])

UCMeta(path='/catalogs/{name}', verb='get')

## Export -

In [28]:
#|hide
from nbdev import nbdev_export
nbdev_export()

InterpolationMissingOptionError: Bad value substitution: option 'lib_name' in section 'DEFAULT' contains an interpolation key 'repo' which is not a valid option name. Raw value: '%(repo)s'

In [ ]:
GhMeta??

Init signature: GhMeta(path, verb, oper_id, summary, doc_url, params, data, preview)
Docstring:      GhMeta(path, verb, oper_id, summary, doc_url, params, data, preview)
Type:           type
Subclasses:     